In [17]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import time
import sys
import tracemalloc

In [2]:
def distance(a,b):
    return np.sqrt(np.sum((a - b)**2))                      

In [3]:
def generate_dataset(N, D):
    return np.random.rand(N, D)

def generate_query(D):
    return np.random.rand(D)

# Naive

In [4]:
def naive_knn(query, dataset, K):
    distances = [distance(query, point) for point in dataset]
    sorted_indices = np.argsort(distances)

    k_sorted_points =  np.array([dataset[i] for i in sorted_indices[:K]])
    k_sorted_distances = np.array(distances)[sorted_indices[:K]]

    return sorted_indices[:K], k_sorted_points,  k_sorted_distances

In [5]:
N,D,K= 1000, 5, 10
ds= generate_dataset(N,D)
q= generate_query(D)
if D==2:
    plt.scatter(ds[:, 0], ds[:, 1])
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.title('Scatter plot of 2D Dataset')
    plt.show()

In [6]:
indices, points, distances=naive_knn(q, ds, K)
print(points.shape)
print(indices)
print(points)
print(distances)

(10, 5)
[371  59 636 632  35 864 720 266 681 958]
[[0.86951383 0.09239464 0.38818843 0.91664639 0.92468818]
 [0.85552405 0.01579039 0.33846085 0.83898189 0.61377044]
 [0.76167036 0.2609945  0.21365706 0.93584771 0.73211638]
 [0.77120675 0.06450603 0.40366192 0.80754863 0.56411609]
 [0.9413087  0.08627977 0.07070335 0.99889504 0.8461975 ]
 [0.93387277 0.02893653 0.10424307 0.91198749 0.60313695]
 [0.62704784 0.04864034 0.49964467 0.92428542 0.83798771]
 [0.74082376 0.16473439 0.18874916 0.78536955 0.55504274]
 [0.61781287 0.04742206 0.09300555 0.93059198 0.74878784]
 [0.97369146 0.0387304  0.46513792 0.7260619  0.52914708]]
[0.14953606 0.19931195 0.26879001 0.28338219 0.28645312 0.30006675
 0.31720142 0.34201022 0.35632783 0.35709567]


# KD Tree

In [7]:
class KDTree:
    def __init__(self, data, leaf_size):
        self.data = np.hstack((data, np.arange(len(data)).reshape(-1, 1))) ##stacking the indices
        self.leaf_size = leaf_size
        self.tree = self.build_kdtree(self.data)
    
    def build_kdtree(self, data, depth=0):
        if len(data) <= self.leaf_size:
            return data  
        
        axis = depth % (data.shape[1] - 1)  # Alternate splitting axis, ignore index column
        sorted_data = data[data[:, axis].argsort()]
        median_index = len(sorted_data) // 2
        left = self.build_kdtree(sorted_data[:median_index], depth + 1)
        right = self.build_kdtree(sorted_data[median_index + 1:], depth + 1)
        
        return (sorted_data[median_index], left, right)
    
    def query(self, query, K):
        indices, distances = self._query(self.tree, query, K, depth=0) 
        points = self.data[indices, :-1]
        return indices, points, distances
    
    def _query(self, node, query, K, depth):
    
        if isinstance(node, np.ndarray): #leaf node is an array
            points = node[:, :-1] 
            original_indices = node[:, -1].astype(int)  
            distances = np.array([distance(query, point) for point in points])
            sorted_indices = np.argsort(distances)
            nearest_indices = original_indices[sorted_indices[:K]]
            nearest_distances = distances[sorted_indices[:K]]
            return nearest_indices, nearest_distances
        
        if isinstance(node, tuple) and len(node) == 3: # non-leaf node is a tuple (median point, left subtree, right subtree)
            median, left, right = node
            axis = depth % (query.shape[0])
            
            if query[axis] < median[axis]:
                primary, other = left, right
            else:
                primary, other = right, left
            
            # Recursively search the primary side
            indices, distances = self._query(primary, query, K, depth + 1)
            
            
            if len(indices) < K or abs(query[axis] - median[axis]) < max(distances): # if the other tree's data also need to be combined
                other_indices, other_distances = self._query(other, query, K, depth + 1)
                combined_indices = np.concatenate([indices, other_indices])
                combined_distances = np.concatenate([distances, other_distances])
                sorted_combined = np.argsort(combined_distances)
                indices = combined_indices[sorted_combined][:K]
                distances = combined_distances[sorted_combined][:K]
            
            return indices, distances
        
       
        return np.array([]), np.array([])




In [9]:

tree = KDTree(ds, leaf_size=20) ## training


indices, points, distances = tree.query(q, K) ##testing

print("Indices of nearest neighbors:", indices)
print("Nearest points:", points)
print("Distances to nearest points:", distances)


Indices of nearest neighbors: [371  59 632  35 864 720 266 681 958 226]
Nearest points: [[0.86951383 0.09239464 0.38818843 0.91664639 0.92468818]
 [0.85552405 0.01579039 0.33846085 0.83898189 0.61377044]
 [0.77120675 0.06450603 0.40366192 0.80754863 0.56411609]
 [0.9413087  0.08627977 0.07070335 0.99889504 0.8461975 ]
 [0.93387277 0.02893653 0.10424307 0.91198749 0.60313695]
 [0.62704784 0.04864034 0.49964467 0.92428542 0.83798771]
 [0.74082376 0.16473439 0.18874916 0.78536955 0.55504274]
 [0.61781287 0.04742206 0.09300555 0.93059198 0.74878784]
 [0.97369146 0.0387304  0.46513792 0.7260619  0.52914708]
 [0.90776489 0.08991513 0.03769023 0.67790346 0.90707521]]
Distances to nearest points: [0.14953606 0.19931195 0.28338219 0.28645312 0.30006675 0.31720142
 0.34201022 0.35632783 0.35709567 0.36787607]


In [ ]:
# ______________________________Remove afer seeing__________________________________


# Measure training time and memory
start_train_time = time.time()
tree = KDTree(ds, leaf_size=20)
end_train_time = time.time()
training_time = end_train_time - start_train_time
training_memory = sys.getsizeof(tree)

print("Training time:", training_time, "seconds")
print("Memory usage after training:", training_memory, "bytes")

# Measure testing time and memory
start_test_time = time.time()
indices, points, distances = tree.query(q, K)
end_test_time = time.time()
testing_time = end_test_time - start_test_time

print("Testing time:", testing_time, "seconds")
print("Indices of nearest neighbors:", indices)
print("Nearest points:", points)
print("Distances to nearest points:", distances)

Training time: 0.011142492294311523 seconds
Memory usage after training: 56 bytes
Testing time: 0.0049855709075927734 seconds
Indices of nearest neighbors: [371  59 632  35 864 720 266 681 958 226]
Nearest points: [[0.86951383 0.09239464 0.38818843 0.91664639 0.92468818]
 [0.85552405 0.01579039 0.33846085 0.83898189 0.61377044]
 [0.77120675 0.06450603 0.40366192 0.80754863 0.56411609]
 [0.9413087  0.08627977 0.07070335 0.99889504 0.8461975 ]
 [0.93387277 0.02893653 0.10424307 0.91198749 0.60313695]
 [0.62704784 0.04864034 0.49964467 0.92428542 0.83798771]
 [0.74082376 0.16473439 0.18874916 0.78536955 0.55504274]
 [0.61781287 0.04742206 0.09300555 0.93059198 0.74878784]
 [0.97369146 0.0387304  0.46513792 0.7260619  0.52914708]
 [0.90776489 0.08991513 0.03769023 0.67790346 0.90707521]]
Distances to nearest points: [0.14953606 0.19931195 0.28338219 0.28645312 0.30006675 0.31720142
 0.34201022 0.35632783 0.35709567 0.36787607]


# LSH 
with flows

In [ ]:
# class LSH:
#     def __init__(self, dataset, num_hashes=5):
#         self.dataset = dataset
#         self.num_hashes = num_hashes
#         self.dataset_augmented = np.hstack((self.dataset, np.ones((self.dataset.shape[0], 1))))  # Adding bias term
#         self.hashes = self._generate_hashes()  # Hashes with bias term included

#     def _generate_hashes(self):
#         return np.random.randn(self.num_hashes, self.dataset.shape[1] + 1)

#     def _hash_(self, point):
#         point_augmented = np.append(point, 1)  
#         return np.sign(np.dot(self.hashes, point_augmented))  

#     def query(self, query, K):
#         query_hash = self._hash_(query)

#         hash_buckets = {}
#         for index, point in enumerate(self.dataset):
#             point_hash = tuple(self._hash_(point))  # Made it into a tuple so it can be used as a dictionary key
#             if point_hash not in hash_buckets:
#                 hash_buckets[point_hash] = []
#             hash_buckets[point_hash].append(index)

#         query_hash_tuple = tuple(query_hash)

#         nearest_neighbors = self.find_k(query, hash_buckets.get(query_hash_tuple, []), K) # looking into same bucket

#         if len(nearest_neighbors) < K: #neighbouring buckets
#             for i in range(self.num_hashes):
#                 if len(nearest_neighbors) >= K:
#                     break
 
#                 modified_query_hash = list(query_hash_tuple)
#                 modified_query_hash[i] = 1 - modified_query_hash[i]  # Flipping the ith hash value
#                 modified_query_hash_tuple = tuple(modified_query_hash)
                
#                 if modified_query_hash_tuple in hash_buckets:
#                     remaining_neighbors = self.find_k(query, hash_buckets[modified_query_hash_tuple], K - len(nearest_neighbors))
#                     nearest_neighbors.extend(remaining_neighbors)

#         nearest_neighbors = sorted(nearest_neighbors, key=lambda x: x[1])[:K] 

#         nearest_indices = [index for index, _ in nearest_neighbors]
#         nearest_points = self.dataset[nearest_indices]
#         nearest_distances = [dist for _, dist in nearest_neighbors]

#         return nearest_indices, nearest_points, nearest_distances

#     def find_k(self, query, bucket_indices, K):
#         distances = []
#         for index in bucket_indices:
#             point = self.dataset[index]
#             dist = distance(query, point) 
#             distances.append((index, dist))

#         return sorted(distances, key=lambda x: x[1])[:K] 


# LSH 

In [19]:
class LSH:
    def __init__(self, dataset, num_hashes=5):
        self.dataset = dataset
        self.num_hashes = num_hashes
        self.dataset_augmented = np.hstack((self.dataset, np.ones((self.dataset.shape[0], 1))))  # Adding bias term
        self.hashes = self._generate_hashes()  # Generate hash functions
        self.hash_buckets = self._create_hash_buckets()  # Create and store hash buckets (Training)

    def _generate_hashes(self):
        return np.random.randn(self.num_hashes, self.dataset.shape[1] + 1)

    def _hash_(self, point):
        point_augmented = np.append(point, 1)  
        return np.sign(np.dot(self.hashes, point_augmented))

    def _create_hash_buckets(self):
        # Create hash buckets by hashing each point in the dataset (part of training)
        hash_buckets = {}
        for index, point in enumerate(self.dataset):
            point_hash = tuple(self._hash_(point))  # Hash each point and store in a tuple
            if point_hash not in hash_buckets:
                hash_buckets[point_hash] = []
            hash_buckets[point_hash].append(index)
        return hash_buckets  # Return the dictionary of hash buckets

    def query(self, query, K):
        # In the query, use the precomputed hash buckets
        query_hash = tuple(self._hash_(query))
        
        # Step 1: Look in the same bucket
        nearest_neighbors = self.find_k(query, self.hash_buckets.get(query_hash, []), K)

        # Step 2: If not enough neighbors, look in neighboring buckets by flipping bits
        if len(nearest_neighbors) < K:
            for i in range(self.num_hashes):
                if len(nearest_neighbors) >= K:
                    break
                modified_query_hash = list(query_hash)
                modified_query_hash[i] = 1 - modified_query_hash[i]  # Flip the i-th hash bit
                modified_query_hash_tuple = tuple(modified_query_hash)
                
                if modified_query_hash_tuple in self.hash_buckets:
                    remaining_neighbors = self.find_k(query, self.hash_buckets[modified_query_hash_tuple], K - len(nearest_neighbors))
                    nearest_neighbors.extend(remaining_neighbors)

        # Sort to ensure top K nearest neighbors
        nearest_neighbors = sorted(nearest_neighbors, key=lambda x: x[1])[:K]
        nearest_indices = [index for index, _ in nearest_neighbors]
        nearest_points = self.dataset[nearest_indices]
        nearest_distances = [dist for _, dist in nearest_neighbors]

        return nearest_indices, nearest_points, nearest_distances

    def find_k(self, query, bucket_indices, K):
        distances = []
        for index in bucket_indices:
            point = self.dataset[index]
            dist = distance(query, point)
            distances.append((index, dist))
        return sorted(distances, key=lambda x: x[1])[:K]


In [ ]:

num_hashes = 15 

lsh = LSH(ds, num_hashes)  ## Training 
nearest_indices, nearest_points, nearest_distances = lsh.query(q, K)  # Testing

print("Query Point:", q)
print(nearest_indices)
print(nearest_points)
print(nearest_distances)

Query Point: [0.88246101 0.0637186  0.3207994  0.88341957 0.79928569]
[371, 919, 241, 289, 155, 243, 834, 411]
[[0.86951383 0.09239464 0.38818843 0.91664639 0.92468818]
 [0.73215984 0.17619877 0.59477978 0.80594397 0.91370886]
 [0.98597543 0.26231166 0.65130941 0.82410741 0.88693687]
 [0.93711472 0.00645282 0.69751127 0.54277892 0.94540154]
 [0.98497918 0.23442819 0.37180185 0.37815929 0.89794973]
 [0.66388381 0.22948614 0.55872825 0.43446548 0.90508384]
 [0.85919142 0.65504589 0.77263651 0.50429122 0.96661007]
 [0.99276238 0.62000479 0.71067845 0.3977653  0.98017266]]
[0.1495360602159034, 0.3597257475076693, 0.41302780898146346, 0.5343820052080215, 0.5543243934141284, 0.5870417679245818, 0.8521161715656228, 0.8615141097391049]


In [21]:
#______________________________Remove after seeing_____________________________

# Number of hash functions
num_hashes = 15 

# Measure training time and memory usage
tracemalloc.start()
start_training_time = time.time()
lsh = LSH(ds, num_hashes)  # Training phase: initializing LSH and generating hashes
end_training_time = time.time()
training_memory, _ = tracemalloc.get_traced_memory()
tracemalloc.stop()
training_time = end_training_time - start_training_time

# Measure testing time and memory usage
tracemalloc.start()
start_testing_time = time.time()
nearest_indices, nearest_points, nearest_distances = lsh.query(q, K)  # Testing phase: querying
end_testing_time = time.time()
testing_memory, _ = tracemalloc.get_traced_memory()
tracemalloc.stop()
testing_time = end_testing_time - start_testing_time

# Convert memory usage to MB
training_memory_mb = training_memory / (1024 * 1024)
testing_memory_mb = testing_memory / (1024 * 1024)

# Output results
print("Training Time:", training_time, "seconds")
print("Training Memory Usage:", training_memory_mb, "MB")
print("Testing Time:", testing_time, "seconds")
print("Testing Memory Usage:", testing_memory_mb, "MB")
print("Query Point:", q)
print("Indices of Nearest Neighbors:", nearest_indices)
print("Nearest Points:", nearest_points)
print("Distances to Nearest Points:", nearest_distances)


Training Time: 0.07346391677856445 seconds
Training Memory Usage: 0.1219949722290039 MB
Testing Time: 0.0 seconds
Testing Memory Usage: 0.0018177032470703125 MB
Query Point: [0.88246101 0.0637186  0.3207994  0.88341957 0.79928569]
Indices of Nearest Neighbors: [371, 59, 636, 632, 35, 864, 266, 269, 760, 805]
Nearest Points: [[0.86951383 0.09239464 0.38818843 0.91664639 0.92468818]
 [0.85552405 0.01579039 0.33846085 0.83898189 0.61377044]
 [0.76167036 0.2609945  0.21365706 0.93584771 0.73211638]
 [0.77120675 0.06450603 0.40366192 0.80754863 0.56411609]
 [0.9413087  0.08627977 0.07070335 0.99889504 0.8461975 ]
 [0.93387277 0.02893653 0.10424307 0.91198749 0.60313695]
 [0.74082376 0.16473439 0.18874916 0.78536955 0.55504274]
 [0.60350425 0.14676321 0.45451395 0.98776797 0.60699443]
 [0.66281775 0.03819977 0.3887972  0.78144778 0.48748142]
 [0.67662317 0.44521113 0.37882831 0.97385696 0.87558704]]
Distances to Nearest Points: [0.1495360602159034, 0.1993119526203916, 0.2687900081947762, 0.2

In [ ]:
#______________________________________Remove after seeing__________________________
tracemalloc.start()  # Start tracking memory

start_test_time = time.time()
indices, points, distances = naive_knn(q, ds, K)
end_test_time = time.time()

current_memory, _ = tracemalloc.get_traced_memory()
tracemalloc.stop()

# Calculate testing time and memory usage
testing_time = end_test_time - start_test_time
testing_memory_mb = current_memory / (1024 * 1024)  # Convert bytes to MB

# Output results
print("Testing Time:", testing_time, "seconds")
print("Testing Memory Usage:", testing_memory_mb, "MB")
print("Indices of Nearest Neighbors:", indices)
print("Nearest Points:", points)
print("Distances to Nearest Points:", distances)

Testing Time: 0.04881477355957031 seconds
Testing Memory Usage: 0.010530471801757812 MB
Indices of Nearest Neighbors: [371  59 636 632  35 864 720 266 681 958]
Nearest Points: [[0.86951383 0.09239464 0.38818843 0.91664639 0.92468818]
 [0.85552405 0.01579039 0.33846085 0.83898189 0.61377044]
 [0.76167036 0.2609945  0.21365706 0.93584771 0.73211638]
 [0.77120675 0.06450603 0.40366192 0.80754863 0.56411609]
 [0.9413087  0.08627977 0.07070335 0.99889504 0.8461975 ]
 [0.93387277 0.02893653 0.10424307 0.91198749 0.60313695]
 [0.62704784 0.04864034 0.49964467 0.92428542 0.83798771]
 [0.74082376 0.16473439 0.18874916 0.78536955 0.55504274]
 [0.61781287 0.04742206 0.09300555 0.93059198 0.74878784]
 [0.97369146 0.0387304  0.46513792 0.7260619  0.52914708]]
Distances to Nearest Points: [0.14953606 0.19931195 0.26879001 0.28338219 0.28645312 0.30006675
 0.31720142 0.34201022 0.35632783 0.35709567]
